# Chlorophyll Background


Notebook on behind-the-scenes construction of `chlorophyll.ipynb`. 


I may refer to and abbreviate the **Python Data Science Handbook** by Jake VanderPlas using 'PDSH', 
possibly followed by a page number or other qualifier. 


Some content herein is broadly applicable to the entire project. 



## Time


See PDSH-189. There are two time mechanisms in play: Python's built-in `datetime` and an improvement called
`datetime64` from **numpy** that enables *arrays* of dates, i.e. time series. 


Consider these two ways of stipulating time slice arguments for `.sel()` applied to a DataSet.
First:  Use a datetime64 with precision to minutes (or finer).
Second: Pass strings that are interpreted as days, inclusive. In pseudo-code: 

```
if do_precision:  
   t0 = dt64('2019-06-01T00:00')
   t1 = dt64('2019-06-01T05:20')
   dss = ds.sel(time=slice(t0, t1))   
else:
    day1 = '24'
    day2 = '27'              # will be 'day 27 inclusive' giving four days of results
    dss = ds.sel(time=slice('2019-06-' + day1, '2019-08-' + day2))

len(dss.time)
```


## Grid of plots

Source: `data_gallery.ipynb`.  


```
rn = range(9); rsi = range(7)

p,a=plt.subplots(3, 3, figsize=(14,14))
plt.rcParams.update({'font.size': 10})
a[0,0].plot(ctdF.time, ctdF.depth, color='r');                                  a[0,0].set(ylim=(200.,0.), title='Depth')
a[0,1].plot(ctdF.time, ctdF.salinity, color='k');                               a[0,1].set(title='Salinity')
a[0,2].plot(ctdF.time, ctdF.temperature, color='b');                            a[0,2].set(title='Temperature')
a[1,0].plot(ctdF.time, ctdF.dissolved_oxygen, color='b');                       a[1,0].set(title='Dissolved Oxygen')
a[1,1].scatter(phF.time.values, phF.ph_seawater.values, color='r');             a[1,1].set(title='pH')
a[1,2].scatter(nitrateF.time.values, nitrateF.scn.values, color='k');           a[1,2].set(title='Nitrate')
a[2,0].plot(parF.time, parF.par_counts_output, color='k');                      a[2,0].set(title='Photosynthetic Light')
a[2,1].plot(fluorF.time, fluorF.fluorometric_chlorophyll_a, color='b');         a[2,1].set(title='Chlorophyll')
a[2,2].plot(siF.time, siF.si0, color='r');                                      a[2,2].set(title='Spectral Irradiance')

a[2,0].text(dt64('2017-08-21T07:30'), 155., 'local midnight', rotation=90, fontsize=15, color='blue', fontweight='bold')
a[2,2].text(dt64('2017-08-21T07:30'), 4.25, 'local midnight', rotation=90, fontsize=15, color='blue', fontweight='bold')

tFmt   = mdates.DateFormatter("%H")                 # an extended format for strftime() is "%d/%m/%y %H:%M"
t0, t1 = ctdF.time[0].values, ctdF.time[-1].values  # establish same time range for each chart
tticks = [dt64('2017-08-21T06:00'), dt64('2017-08-21T12:00'), dt64('2017-08-21T18:00')]

for i in rn: j, k = i//3, i%3; a[j, k].set(xlim=(t0, t1),xticks=tticks); a[j, k].xaxis.set_major_formatter(tFmt)
print('')
```

In [ ]:
# mini-source control: Last copied 29-SEP-2020: to tilt*, chlorophyll*, rca*, argo*
#                      last revised 09-OCT-2020
import os, sys, time, glob

from IPython.display import clear_output             # use inside loop with clear_output(wait = True) followed by print(i)
import warnings                                      # use with warnings.filterwarnings('ignore') or 'once'

home_dir = os.getenv("HOME")
this_dir = home_dir + '/chlorophyll/'
data_dir = '/data/'
data1_dir = '/data1'

from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors
import numpy as np, pandas as pd, xarray as xr
from numpy import datetime64 as dt64, timedelta64 as td64

def doy(theDatetime): return 1 + int((theDatetime - dt64(str(theDatetime)[0:4] + '-01-01')) / td64(1, 'D')) # 1, 2, .... , 365, [366]
def dt64_from_doy(year, doy): return dt64(str(year) + '-01-01') + td64(doy-1, 'D')
def day_of_month_to_string(d): return str(d) if d > 9 else '0' + str(d)

print('\nJupyter Notebook running Python {}'.format(sys.version_info[0]))

### Two ways of stipulating time slice arguments for a Dataset .sel()

- First uses a datetime64 and is precise to minutes (or finer)
- Second passes strings that are interpreted as days, inclusive

```
if do_precision:  
    t0 = dt64('2019-06-01T00:00')
    t1 = dt64('2019-06-01T05:20')
    dss = ds.sel(time=slice(t0, t1))   
else:
    day1 = '24'
    day2 = '27'              # will be 'day 27 inclusive' giving four days of results
    dss = ds.sel(time=slice('2019-06-' + day1, '2019-08-' + day2))
len(dss.time)
```

# Spectrophotometer


Code produces one month of water column profiles.


For any instrument one must discern: 'When is this making measurements? What is measured?  Frequency?'


For example the SP runs on ascent at about 3.7 samples per second; whereas nitrate runs on ascent
at about 3 samples per minute. Going from an unexamined data file to an accurate picture of how the
sensor works is a bit of effort. 


## deconstruct data patterns

Needs work...


- Start by assigning the data to an XArray Dataset using `ds = xr.open_dataset(fnm)`. 
    - When the data are dispersed across multiple files use `ds = xr.open_mfdataset('data_with_*_.nc')`
        - Note this uses wildcard construction 
- Examine the Dataset using `ds`
    - Look at the time dimension
        - It is sometimes a poor choice to use `obs` or `observation` as the dimensional coordinate
            - This creates degeneracy over multiple files
            - We can use `.swap_dims` to swap time in; making multiple Datasets combinable
    - `ds.time[0].values, ds.time[-1].values` gives a span but nothing about duty cycles
        - As an example: 2019 spectrophotometer data
            - Oregon Slope Base
            - 86 channels
            - 7 million samples


. . . some work happens leading to . . . 

            - Only operates during midnight and noon ascent; at 3.7 samples per second
            - Data has frequent dropouts over calendar time
            - Data has spikes that tend to register across all 86 channels
            - Very poor documentation; even the SME report is cursory
            

- Next examine an individual day (two ascent profiles).


```
# revealing looks... isolate one to print
# ds_ascent2.time[0], ds_ascent2.time[-1]
# ds_ascent2.optical_absorption
# ds_ascent2.beam_attenuation
# ds_optaa
```

## ([`optaa`](https://oceanobservatories.org/instrument-class/optaa/)) at **Oregon Slope Base**, 2019, JAN


<BR>
<img src="./images/spectrophotometer/osb_2019_sp_availability.png" style="float: left;" alt="drawing" width="1200"/>
<div style="clear: left"><BR>
    
    
Above: Spectrophotometer data availability at Oregon Slope Base site courtesy the Interactive Oceans data portal. 
2019 is somewhat intermittent and stops in September. The first run here will consolidate 2019 data; and later 
expand to include the full duration and all sites. 
    
    
### Exposition


* Instrument duty cycle
    * The spectrophotometer sensor located on the Oregon Slope Base shallow profiler is *not* on all the time.
    * It runs episodically, twice per day: During the midnight and noon profile ascents (and *not* during descent)
        * **The first profile is 7:22 Zulu: midnight off the coast of Oregon state**
        * RCA shallow profilers execute nine profiles per day from 200m to 5m nominal depths, then back down
        * Foreshadowing: A scientist *could* consider a single profile as a time-evolving study since the 
          rise-fall requires over an hour to complete. In this work we treat the profile as water column 
          characterization with a timestamp. In other words the operative dimension is pressure/depth, not time; 
          and profiles are then considered in coarser time series.
    * Two of the nine daily profiles run at local midnight and noon
        * These feature spectrophotometer measurements during *ascent* and nitrate measurements on *descent*.
    * These two special profiles take upwards of two hours to complete. (The other seven require less time.)
    * In between profiles: The profiler or Science Pod (SCIP) is at rest on a platform at a depth of 200 meters
    * The ascent minimum depth is five meters but is typically more, varying with sea conditions
* Spectrophotometer (two ascents per day, local midnight and noon as noted above)
    * Data are optical absorption (abbreviated **OA**), beam attenuation (abbreviated **BA**), time and pressure
        * In this work pressure in dbar and depth in meters are treated as equivalent.
    * Instrument sampling rate is ~3.7 samples per second
    * Instrument records 86 spectral channels
        * Light wavelength is ~(400nm + channel number x 4nm)
        * Channel width is ~20nm so channels overlap
        * Signals shift with wavelength making it possible to stack profiles in a single chart
    * Channels 0, 83, 84 and 85 tend to give `nan` values (not usable) for both OA and BA
        * In this work we tend to use channels 2 through 82
    * Both OA and BA data are idiosyncratic
        * The midnight OA data are quantized in a peculiar manner; see charts below
        * The noon OA are *somewhat* quantized but have more reasonable / data-like structure
        * BA data are not fraught with the OA quantization issue
            * Both midnight and noon BA data include substantial noise
            * Variance is also apparent in BA data
            * This suggests filtering by depth bin and possibly discarding outliers
* Un-answered questions
    * Why are OA data different in midnight versus noon profiles? 
    * Are OA and BA typically combined into a turbidity value?
    * What wavelength ranges are of particular interest?
    * How do these signals compare with fluorometers, nitrate, CTD, pH, etcetera? 
        * The OOI site has a brief SME evaluation circa 2016 that does not illuminate actual data use cases.
* References froom OOI
    * [Table of instruments / designators / locations](https://oceanobservatories.org/instrument-series/optaad/)
    * [Spectrophotometer page](https://oceanobservatories.org/instrument-class/optaa/)
    * [Subject Matter Expert evaluation](https://oceanobservatories.org/2016/07/successful-sme-evaluation-spectrophotometer-optaa/)
    * [Code](https://github.com/oceanobservatories/ion-functions/blob/master/ion_functions/data/opt_functions.py)


Paraphrasing the Subject Matter Export evaluation (link above): 


> Dr. Boss (SME) verified 1.5 months of data (April-May 2015): Processing and plotting data using the raw data and vendor calibration files 
> from the AC-S, salinity and temperature from a collocated CTD data to correct absorption and attenuation median spectra and scattering, 
> and data from a collocated fluorometer to cross-check the chlorophyll and POC results.
> 
> Consistency between the sensors suggests that they did not foul during the deployment. Not only did his results show that accurate data 
> was being produced by all the sensors in question, but the AC-S (an extremely sensitive instrument normally deployed for very short periods
> of time) did not drift noticeably during the deployment period, a notable achievement.



From the [sheet on Optical Absorption (**OA**)](https://oceanobservatories.org/wp-content/uploads/2015/10/1341-00700_Data_Product_SPEC_OPTABSN_OOI.pdf):


> The primary instrument (OPTAA) is the WET Labs ac-s spectral absorption and attenuation meter. 
The instrument provides a 75 wavelength output from approximately 400–750 nm with approximately 
4 nm steps. Individual filter steps have a full-width half maximum response that
range between about 10 to 18 nm. 
>
> There are a total of 35 OPTAA instruments deployed
throughout the initial OOI construction and integrated into the Pioneer, Endurance, Regional and
Global arrays. They are deployed at fixed depths (near-surface, mid-water column and sea floor)
and installed on moored profilers.
>
> The ac-s performs concurrent measurements of the water attenuation and absorption 
(the latter called 'OPTABSN').
>
> OPTABSN is a L2 product in that computation requires the raw signals emanating from a properly
calibrated and configured instrument as well as water temperature (TEMPWAT) and practical
salinity (PRACSAL) derived from a co-located and synchronized CTD. 
>
> While small corrections
for salinity are available at visible wavelengths (< 700 nm), temperature and salinity corrections
are more significant at infrared wavelengths (> 700 nm) and must be performed on both the
absorption and attenuation (OPBATTN) signals.


The [beam attenuation (**BA**) sheet](https://oceanobservatories.org/wp-content/uploads/2015/10/1341-00690_Data_Product_SPEC_OPTATTN_OOI.pdf)
is similar. Both give a mathematical basis for the data as well as (MATLAB?) code. 


### Practical interpretation of Spectrophotometer data
    
    
The data are provided as NetCDF-CF format files. The code used here is the Python `XArray` package 
built to work with this file format, specifically via the `Dataset` and 
the `DataArray` structures. These in turn inherit from the `pandas` `DataFrame` and `Series`; which in turn
are built on the `numpy` n-dimensional array `ndarray`. All of this requires considerable time to 
learn and internalize. Some care is taken in this narrative to provide salient remarks for the Learner. 
In polemical terms I recommend two approaches for the person new to this set of tools: 
    
    
- Read through this notebook without too much concern for detail (suited to a focus on 
    the data, not on learning to build new workflows).
- Work methodically through Jake VanDerplas' excellent (free, online) book 
    [_The Python Data Science Handbook_ (herein abbreviated **PDSH**)](https://jakevdp.github.io/PythonDataScienceHandbook/) 
    particularly chapters 2 and 3, to gain expertise with `NumPy` and `pandas` prior to taking on `XArray`. 


As an example of the challenge of learning `XArray`: The reduction of this data to binned profiles
requires a non-trivial workflow. A naive approach can result in a calculation that should take 
a seconds run for hours. (A key idea of this workflow -- the sortby() step -- is found on page 137 of **PDSH**.)
    
    
- `swap_dims()` to substitute `pressure` for `time` as the ordinate dimension
- `sortby()` to make the `pressure` dimension monotonic
- Create a pressure-bin array to guide the subsequent data reduction
- `groupby_bins()` together with `mean()` to reduce the data to a 0.25 meter quantized profile
- use `transpose()` to re-order wavelength and pressure, making the resulting `DataArray` simpler to plot
- accumulate these results by day as a list of `DataArrays`
- From this list create an `XArray Dataset`
- Write this to a new NetCDF file

In [ ]:
####################
#
# Spectrophotometer
#   OA = Optical Absorbance (deferred owing to pathologies in the data, particularly midnight)
#   BA = Beam Absorbance
#
####################

# single data read covers OA and BA, both ascents each day, 2019 JAN - SEP
ds_optaa = xr.open_dataset(data_dir + 'rca/simpler/osb_sp_optaa_2019.nc')

warnings.filterwarnings('ignore')

include_plots = False

m_strs = ['01', '02', '03', '04', '05', '06', '07', '08', '09']           # relevant 2019 months
m_days = [31, 28, 31, 30, 31, 30, 31, 31, 30]                             # days per month in 2019

month_index = 0                                                           # manage time via months and days; 0 is January
month_str   = m_strs[month_index]  
year_str    = '2019'

n_meters          = 200
n_bins_per_meter  = 4
halfbin           = (1/2) * (1/n_bins_per_meter)
n_pressure_bins   = n_meters * n_bins_per_meter
n_wavelengths     = 86
wavelength        = [i for i in range(n_wavelengths)]
p_bounds          = np.linspace(0., n_meters, n_pressure_bins + 1)             # 801 bounds: 0., .25, ..., 200.                   
pressure          = np.linspace(halfbin, n_meters - halfbin, n_pressure_bins)  # 800 centers: 0.125, ..., 199.875                  
oa_upper_bound    = 40.
ba_upper_bound    = 0.7
wavelength_stride = 8

ndays = m_days[month_index]
ndayplots, dayplotdays = 3, [0, 10, 20]

l_da_oa_midn, l_da_oa_noon, l_da_ba_midn, l_da_ba_noon = [], [], [], []       # these lists accumulate DataArrays by day

if include_plots:
    fig_height, fig_width, fig_n_across, fig_n_down = 6, 6, 4, ndayplots
    fig, axs = plt.subplots(ndayplots, fig_n_across, figsize=(fig_width * fig_n_across, fig_height*fig_n_down), tight_layout=True)

# for day_index in range(m_days[month_index]):                                   # loop: days of a chosen month 
for day_index in range(ndays):
    
    day_str  = day_of_month_to_string(day_index + 1); date_str = year_str + '-' + month_str + '-' + day_str
    this_doy = doy(dt64(date_str))
    clear_output(wait = True); print("on day", day_str, 'i.e. doy', this_doy)
    midn_start = date_str + 'T07:00:00'
    midn_done  = date_str + 'T10:00:00'
    noon_start = date_str + 'T20:00:00'
    noon_done  = date_str + 'T23:00:00'

    # pull out OA and BA for both midnight and noon ascents; and swap in pressure for time
    ds_midn = ds_optaa.sel(time=slice(dt64(midn_start), dt64(midn_done))).swap_dims({'time':'int_ctd_pressure'})
    ds_noon = ds_optaa.sel(time=slice(dt64(noon_start), dt64(noon_done))).swap_dims({'time':'int_ctd_pressure'})
    
    da_oa_midn = ds_midn.optical_absorption.expand_dims({'doy':[this_doy]})
    da_oa_noon = ds_noon.optical_absorption.expand_dims({'doy':[this_doy]})
    da_ba_midn = ds_midn.beam_attenuation.expand_dims({'doy':[this_doy]})
    da_ba_noon = ds_noon.beam_attenuation.expand_dims({'doy':[this_doy]})
    
    del da_oa_midn['time']; del da_oa_noon['time']; del da_ba_midn['time']; del da_ba_noon['time']
    
    l_da_oa_midn.append(da_oa_midn.sortby('int_ctd_pressure').groupby_bins("int_ctd_pressure", p_bounds, labels=pressure).mean().transpose('wavelength', 'int_ctd_pressure_bins', 'doy'))
    l_da_oa_noon.append(da_oa_noon.sortby('int_ctd_pressure').groupby_bins("int_ctd_pressure", p_bounds, labels=pressure).mean().transpose('wavelength', 'int_ctd_pressure_bins', 'doy'))
    l_da_ba_midn.append(da_ba_midn.sortby('int_ctd_pressure').groupby_bins("int_ctd_pressure", p_bounds, labels=pressure).mean().transpose('wavelength', 'int_ctd_pressure_bins', 'doy'))
    l_da_ba_noon.append(da_ba_noon.sortby('int_ctd_pressure').groupby_bins("int_ctd_pressure", p_bounds, labels=pressure).mean().transpose('wavelength', 'int_ctd_pressure_bins', 'doy'))
    
    if include_plots and day_index in dayplotdays:      # if this is a plotting day: Add to the chart repertoire
        
        dayplotindex = dayplotdays.index(day_index) 
        oa_plot_wavelength = 12
                
        axs[dayplotindex][0].scatter(l_da_oa_midn[-1][oa_plot_wavelength], pressure,  marker=',', s=1, color='k') 
        axs[dayplotindex][1].scatter(l_da_oa_noon[-1][oa_plot_wavelength], pressure,  marker=',', s=1, color='b') 
        axs[dayplotindex][0].set(xlim = (.0, oa_upper_bound), ylim = (200., 0.), title='OA midnight')
        axs[dayplotindex][1].set(xlim = (.0, oa_upper_bound), ylim = (200., 0.), title='OA noon')

        for chan_sel_index in range(2, 83, wavelength_stride):
            
            axs[dayplotindex][2].plot(l_da_ba_midn[-1][chan_sel_index], pressure,  marker='', color='r') 
            axs[dayplotindex][3].plot(l_da_ba_noon[-1][chan_sel_index], pressure,  marker='', color='g')
            axs[dayplotindex][2].set(xlim = (.0, ba_upper_bound), ylim = (200., 0.), title='BA midnight')
            axs[dayplotindex][3].set(xlim = (.0, ba_upper_bound), ylim = (200., 0.), title='BA noon')

        # Superimpose raw to compare: OA shows quantization; BA shows noise, outliers and suspicious jumps
        axs[dayplotindex][0].scatter(ds_midn.optical_absorption.isel(wavelength=oa_plot_wavelength), ds_midn.int_ctd_pressure, marker=',', s=1., color='r'); 
        axs[dayplotindex][1].scatter(ds_noon.optical_absorption.isel(wavelength=oa_plot_wavelength), ds_noon.int_ctd_pressure, marker=',', s=1., color='g'); 
        axs[dayplotindex][2].scatter(ds_midn.beam_attenuation.isel(wavelength = 2 + wavelength_stride), ds_midn.int_ctd_pressure, marker=',', s=1., color='b'); 
        axs[dayplotindex][3].scatter(ds_noon.beam_attenuation.isel(wavelength = 2 + wavelength_stride), ds_noon.int_ctd_pressure, marker=',', s=1., color='k'); 

save_figure = False
if save_figure: fig.savefig('/home/ubuntu/savefig.png')

    
########################################
# 
# save resulting datasets for optaa
#
########################################

save_optaa_datasets = False

if save_optaa_datasets: 

    ds_oa_midn = xr.concat(l_da_oa_midn, dim="doy").to_dataset(name='optical_absorption')
    ds_oa_noon = xr.concat(l_da_oa_noon, dim="doy").to_dataset(name='optical_absorption')
    ds_ba_midn = xr.concat(l_da_ba_midn, dim="doy").to_dataset(name='beam_attenuation')
    ds_ba_noon = xr.concat(l_da_ba_noon, dim="doy").to_dataset(name='beam_attenuation')

    ds_oa_midn.to_netcdf("/data1/optaa/oa_midn_2019_01.nc")
    ds_oa_noon.to_netcdf("/data1/optaa/oa_noon_2019_01.nc")
    ds_ba_midn.to_netcdf("/data1/optaa/ba_midn_2019_01.nc")
    ds_ba_noon.to_netcdf("/data1/optaa/ba_noon_2019_01.nc")

# nitrate 

## profile dataset builder

This code follows suit the spectrophotometer. It is simpler because there is only a nitrate value 
and no wavelength channel. 

I kept the pressure bins the same even though the nitrate averates about 3 three samples or less per minute
during a 70 minute ascent. That's about three meters per minute so one sample per meter. Since the 
spectrophotometer bin depth is 0.25 meters there are necessarily a lot of empty bins (bins with no data)
for the nitrate profile. 

## two open issues


A curious artifact of the situation is from a past bias: I had understood that the SCIP makes pauses 
on descent to accommodate the nitrate sensor. I may be in error but now it looks like this sensor, 
the nitrate sensor, is observing on ascent which is continuous. This leaves open the question of 
why the pauses occur on the descent. If I have that right. 


Finally there are two nitrate signals: 'samp' and 'dark'. This code addresses only 'samp' as 'dark'
is showing nothing of interest. So this is an open issue.

In [ ]:
####################
#
# Nitrate
# 
#   dims:       time
#   coords:     time and int_ctd_pressure
#   data array: nitrate concentration
#
# To do
#   identify when the data happens
#   verify that the 'dark' means nothing...
# 
####################

ds_n03dark = xr.open_dataset("/data/rca/simpler/osb_sp_nutnr_a_dark_2019.nc")
ds_n03samp = xr.open_dataset("/data/rca/simpler/osb_sp_nutnr_a_sample_2019.nc")

warnings.filterwarnings('ignore')

include_charts = False

m_strs = ['01', '02', '03', '04', '05', '06', '07', '08', '09']           # relevant 2019 months
m_days = [31, 28, 31, 30, 31, 30, 31, 31, 30]                             # days per month in 2019

month_index = 0                                                           # manage time via months and days; 0 is January
month_str   = m_strs[month_index]  
year_str    = '2019'

n_meters          = 200
n_bins_per_meter  = 4
halfbin           = (1/2) * (1/n_bins_per_meter)
n_pressure_bins   = n_meters * n_bins_per_meter
p_bounds          = np.linspace(0., n_meters, n_pressure_bins + 1)             # 801 bounds: 0., .25, ..., 200.                   
pressure          = np.linspace(halfbin, n_meters - halfbin, n_pressure_bins)  # 800 centers: 0.125, ..., 199.875                  
nc_upper_bound    = 40.

ndays = m_days[month_index]
ndayplots, dayplotdays = 10, list(range(10))

l_da_nc_midn, l_da_nc_noon = [], []       # these lists accumulate DataArrays by day

if include_charts:
    fig_height, fig_width, fig_n_across, fig_n_down = 4, 4, 2, ndayplots
    fig, axs = plt.subplots(ndayplots, fig_n_across, figsize=(fig_width * fig_n_across, fig_height * fig_n_down), tight_layout=True)

for day_index in range(ndays):
    
    day_str  = day_of_month_to_string(day_index + 1); date_str = year_str + '-' + month_str + '-' + day_str
    this_doy = doy(dt64(date_str))
    clear_output(wait = True); print("on day", day_str, 'i.e. doy', this_doy)
    midn_start = date_str + 'T07:00:00'
    midn_done  = date_str + 'T10:00:00'
    noon_start = date_str + 'T20:00:00'
    noon_done  = date_str + 'T23:00:00'

    # pull out OA and BA for both midnight and noon ascents; and swap in pressure for time
    ds_midn = ds_n03samp.sel(time=slice(dt64(midn_start), dt64(midn_done))).swap_dims({'time':'int_ctd_pressure'})
    ds_noon = ds_n03samp.sel(time=slice(dt64(noon_start), dt64(noon_done))).swap_dims({'time':'int_ctd_pressure'})
    
    # print('pressures:', ds_midn.int_ctd_pressure.size, ds_noon.int_ctd_pressure.size, '; times:', ds_midn.time.size, ds_noon.time.size)    
    midn = True if ds_midn.time.size > 0 else False
    noon = True if ds_noon.time.size > 0 else False
        
    if midn:
        da_nc_midn = ds_midn.nitrate_concentration.expand_dims({'doy':[this_doy]})
        del da_nc_midn['time']
        l_da_nc_midn.append(da_nc_midn.sortby('int_ctd_pressure').groupby_bins("int_ctd_pressure", p_bounds, labels=pressure).mean().transpose('int_ctd_pressure_bins', 'doy'))
        
    if noon:
        da_nc_noon = ds_noon.nitrate_concentration.expand_dims({'doy':[this_doy]})
        del da_nc_noon['time']
        l_da_nc_noon.append(da_nc_noon.sortby('int_ctd_pressure').groupby_bins("int_ctd_pressure", p_bounds, labels=pressure).mean().transpose('int_ctd_pressure_bins', 'doy'))

    if include_charts and day_index in dayplotdays:      # if this is a plotting day: Add to the chart repertoire

        dayplotindex = dayplotdays.index(day_index) 

        if midn:
            axs[dayplotindex][0].scatter(l_da_nc_midn[-1], pressure,  marker=',', s=2., color='r') 
            axs[dayplotindex][0].set(xlim = (.0, nc_upper_bound), ylim = (200., 0.), title='NC midnight')
            axs[dayplotindex][0].scatter(ds_midn.nitrate_concentration, ds_midn.int_ctd_pressure, marker=',', s=1., color='b'); 
            
        if noon:
            axs[dayplotindex][1].scatter(l_da_nc_noon[-1], pressure,  marker=',', s=2., color='g')
            axs[dayplotindex][1].set(xlim = (.0, nc_upper_bound), ylim = (200., 0.), title='NC noon')
            axs[dayplotindex][1].scatter(ds_noon.nitrate_concentration, ds_noon.int_ctd_pressure, marker=',', s=1., color='k'); 

save_figure = False
if save_figure: fig.savefig('/home/ubuntu/chlorophyll/images/misc/nitrate_2019_JAN_1_to_10.png')

save_nitrate_profiles = False

if save_nitrate_profiles: 
    ds_nc_midn = xr.concat(l_da_nc_midn, dim="doy").to_dataset(name='nitrate_concentration')
    ds_nc_noon = xr.concat(l_da_nc_noon, dim="doy").to_dataset(name='nitrate_concentration')

    ds_nc_midn.to_netcdf("/data1/nutnr/nc_midn_2019_01.nc")
    ds_nc_noon.to_netcdf("/data1/nutnr/nc_noon_2019_01.nc")

# pH profile builder

`phsen` is pH

# Fluorescence profile builder

`flort` is chlorophyll concentration

# Photosynthetically Available Radiation (PAR) profile builder

`parad` is sunlight available for photosynthesis

# Spectral irradiance profile builder

`spkir` is also sunlight, specifically ?-welling spectral irradiance

# CTD profile builder

Here we return to 9 profiles per day: `ctdpf` is salinity, temperature and depth measured on all profiles.

## ---

# Learning basics

## Sandbox build up to XArray from NumPy ndarray to pandas DataFrame

### numpy ndarrays 

* do not have row and column headers; whereas pandas DataFrames do have typed headers
* indexing has an equivalence of `[2][0]` to `[2,0]` 
    * The latter (with comma) is the presented way in PDSH
    * This duality does not work for DataFrames
* has row-then-column index order...
    * ....with three rows in `[['l','i','s','t','1'],['s','c','n','d','2'],['t','h','r','d','3']]` 
* has slice by dimension as `start:stop:step` by default `0, len (this dimension), 1` 
    * ...exception: when `step` is negative `start` and `stop` are reversed
    * ...multi-dimensional slices separated by commas
    

### pandas DataFrames

* constructor takes `data=<ndarray>` and both `index` and `columns` arguments... 
    * ...2 dimensions only: higher dimensions and they say 'use XArray'
    * ...and switching required a `.T` transpose
* indexing by column and row header values, separated as in `[column_header][row_header]`
    * as this reverses order from ndarrays: Better confirm... seems to be the case
    * skip index/columns: defaults to integers.
    
    
#### problem and solution

* Problem: I can process a Dataset in a matter of hours that should take seconds
    * I have observations across 86 channels. Call that one observation with 86 values.
    * I have both a depth and a time associated with each observation. 
    * As a result I have multiply-indexed data.
    * `time` is the default dimension; but I am interested in sorting by depth
        * In fact by depth bins that incorporate many observations
    * There are 86 x 14000 data values to average into 86 x 800 bins
        * Expected time is perhaps a few seconds
        * Actual time using 'self-evident' methods is an hour or more
* Solution
    * Page 137 of PDSH begins a section on **Rearranging Multi-Indices**
    * The first sub-heading is **Sorted and unsorted indices**
    * The text after both of these headings is instructive so I will quote directly.
    
> ***Rearranging Multi-indices***<BR>
One of the keys to working with multiply indexed data is knowing how to effectively transform the data. 
There are a number of operations that will preserve all the information in the dataset, but rearrange 
it for the purposes of various computations. [...] There are many [ways] to finely control the rearrangement
of data between heirarchical indices and columns.
    
> ***Sorted and unsorted indices***<BR>
Earlier, we briefly mentioned a caveat, but we should emphasize it more here. 
*Many of the `MultiIndex`slicing operations will fail if the index is not sorted.*

In [5]:
###################
#
# A micro study of ndarray to DataFrame translation
#
###################

import numpy as np
import pandas as pd

# Here is an ndarray construction from a built list of lists (not used in what follows): 
# arr = np.array([range(i, i+5) for i in [2, 4, 6]])                                       # the range() runs across columns; 2 4 6 are rows

# ndarray construction: Notice all list elements are of the same type (strings)
arr = np.array([['l','i','s','t','1'],['s','c','n','d','2'],['t','h','r','d', '3']])

print('\nndarray from a list of lists (notice no comma delimiter):\n\n', arr, '\n\nand indexing comparison: first [0][2] then [2][0]:', arr[0][2], arr[2][0]) 
print('\nand tuplesque indexing [0, 2] or [2, 0] equivalently gives:', arr[0,2], arr[2,0])
print('\nSo ndarrays index [slow][fast] equivalent to [row][column]\n')

rowlist=["2row", "4row", "6row"]
columnlist = ["col_a", "col_b", "col_c", "col_d", "col_e"]
df = pd.DataFrame(data=arr, index=rowlist, columns=columnlist)

print(df, '\n\nis a DataFrame from the ndarray; so now index ["col_c"]["6row"]:', df['col_c']['6row'])

df = pd.DataFrame(data=arr.T, index=columnlist, columns=rowlist)

print('\nHere is a Dataframe from a transpose of the ndarray\n\n', df, '\n\nindexing 2row then col_e:', df['2row']['col_e'])
print('\nSo the column of a DataFrame is indexed first, then the row: Reverses the sense of the 2D ndarray.\n')
print('Now skipping the index= argument so the row labels default to integers:\n')

df = pd.DataFrame(data=arr, columns=columnlist)

print(df, '\n\n...so now indexing ["col_d"][0]:', df['col_d'][0], '\n')

df = pd.DataFrame(data=arr, index=rowlist)

print(df, '\n\nhaving done it the other way: used index= but not columns=. Here is element [0]["4row"]:', df[0]['4row'])


ndarray from a list of lists (notice no comma delimiter):

 [['l' 'i' 's' 't' '1']
 ['s' 'c' 'n' 'd' '2']
 ['t' 'h' 'r' 'd' '3']] 

and indexing comparison: first [0][2] then [2][0]: s t

and tuplesque indexing [0, 2] or [2, 0] equivalently gives: s t

So ndarrays index [slow][fast] equivalent to [row][column]

     col_a col_b col_c col_d col_e
2row     l     i     s     t     1
4row     s     c     n     d     2
6row     t     h     r     d     3 

is a DataFrame from the ndarray; so now index ["col_c"]["6row"]: r

Here is a Dataframe from a transpose of the ndarray

       2row 4row 6row
col_a    l    s    t
col_b    i    c    h
col_c    s    n    r
col_d    t    d    d
col_e    1    2    3 

indexing 2row then col_e: 1

So the column of a DataFrame is indexed first, then the row: Reverses the sense of the 2D ndarray.

Now skipping the index= argument so the row labels default to integers:

  col_a col_b col_c col_d col_e
0     l     i     s     t     1
1     s     c     n     d   